# 阶段2：宏观经济SubAgent原型

**目标**：实现宏观经济分析SubAgent，验证DeepAgents架构可行性

**架构**：DeepAgents框架（Main Agent + SubAgent）

**工具**：
- AKShare数据工具：GDP、CPI、PMI
- 知识库检索工具：向量检索、JSON查询


## 工具函数定义

**AKShare工具**：获取宏观经济数据（GDP、CPI、PMI）

**知识库检索工具**：向量检索和JSON查询


In [ ]:
# ========== AKShare工具函数 ==========
import akshare as ak
import pandas as pd
from typing import Optional
import logging

# 配置日志 - 使用WARNING级别避免刷屏
logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

# 强制关闭干扰日志（确保流式输出纯净）
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("httpcore").setLevel(logging.WARNING)
logging.getLogger("sentence_transformers").setLevel(logging.WARNING)
logging.getLogger("chromadb").setLevel(logging.WARNING)
logging.getLogger("akshare").setLevel(logging.WARNING)


def get_gdp_quarterly() -> str:
    """获取季度GDP数据

    Returns:
        格式化的GDP数据字符串

    Raises:
        Exception: 数据获取失败时返回错误信息
    """
    print("[工具] 获取GDP数据...", end="", flush=True)
    try:
        df = ak.macro_china_gdp()
        if df.empty:
            print(" 失败")
            return "获取GDP数据失败：数据为空"

        df = df.rename(columns={
            '季度': 'quarter',
            '国内生产总值-绝对值': 'gdp',
            '国内生产总值-同比增长': 'gdp_yoy'
        })
        df = df[['quarter', 'gdp', 'gdp_yoy']]
        df['gdp'] = pd.to_numeric(df['gdp'], errors='coerce')
        df['gdp_yoy'] = pd.to_numeric(df['gdp_yoy'], errors='coerce')
        df = df.dropna().reset_index(drop=True)

        result = "GDP季度数据(最近5个季度):\n\n"
        for _, row in df.head(5).iterrows():
            result += f"季度: {row['quarter']}, GDP: {row['gdp']}亿元, 同比增长: {row['gdp_yoy']}%\n"

        print(" 完成")
        return result
    except Exception as e:
        print(f" 失败: {e}")
        return f"获取GDP数据失败: {str(e)}"


def get_cpi_monthly() -> str:
    """获取月度CPI数据

    Returns:
        格式化的CPI数据字符串

    Raises:
        Exception: 数据获取失败时返回错误信息
    """
    print("[工具] 获取CPI数据...", end="", flush=True)
    try:
        df = ak.macro_china_cpi()
        if df.empty:
            print(" 失败")
            return "获取CPI数据失败：数据为空"

        df = df.rename(columns={
            '月份': 'month',
            '全国-同比增长': 'cpi_yoy',
            '全国-环比增长': 'cpi_mom',
            '全国-累计': 'cpi_ytd'
        })
        df = df[['month', 'cpi_yoy', 'cpi_mom', 'cpi_ytd']]
        df['cpi_yoy'] = pd.to_numeric(df['cpi_yoy'], errors='coerce')
        df['cpi_mom'] = pd.to_numeric(df['cpi_mom'], errors='coerce')
        df['cpi_ytd'] = pd.to_numeric(df['cpi_ytd'], errors='coerce')
        df = df.dropna().reset_index(drop=True)

        result = "CPI月度数据(最近6个月):\n\n"
        for _, row in df.head(6).iterrows():
            result += f"月份: {row['month']}, 同比: {row['cpi_yoy']}%, 环比: {row['cpi_mom']}%, 累计: {row['cpi_ytd']}%\n"

        print(" 完成")
        return result
    except Exception as e:
        print(f" 失败: {e}")
        return f"获取CPI数据失败: {str(e)}"


def get_pmi_manufacturing() -> str:
    """获取制造业PMI数据

    Returns:
        格式化的PMI数据字符串

    Raises:
        Exception: 数据获取失败时返回错误信息
    """
    print("[工具] 获取PMI数据...", end="", flush=True)
    try:
        df = ak.macro_china_pmi()
        if df.empty:
            print(" 失败")
            return "获取PMI数据失败：数据为空"

        df = df.rename(columns={
            '月份': 'month',
            '制造业-指数': 'pmi',
            '制造业-同比增长': 'pmi_yoy'
        })
        df = df[['month', 'pmi', 'pmi_yoy']]
        df['pmi'] = pd.to_numeric(df['pmi'], errors='coerce')
        df['pmi_yoy'] = pd.to_numeric(df['pmi_yoy'], errors='coerce')
        df = df.dropna().reset_index(drop=True)

        result = "PMI制造业数据(最近6个月):\n\n"
        for _, row in df.head(6).iterrows():
            result += f"月份: {row['month']}, PMI指数: {row['pmi']}, 同比增长: {row['pmi_yoy']}%\n"

        print(" 完成")
        return result
    except Exception as e:
        print(f" 失败: {e}")
        return f"获取PMI数据失败: {str(e)}"


# ========== 知识库检索工具 ==========
# 注意：当前在notebook中直接定义KnowledgeRetriever，与src/analyst_chain/tools/knowledge_retriever.py中的代码重复
# 未来拆分到src/时，应统一使用src/中的版本，notebook改为从src/导入
import json
from pathlib import Path
from langchain_chroma import Chroma
from langchain_huggingface import HuggingFaceEmbeddings


class KnowledgeRetriever:
    """知识库检索工具"""

    def __init__(self,
                 vector_db_path: str = "../data/processed/knowledge/vector_db/macro_economy",
                 json_dir_path: str = "../data/processed/knowledge/structured/macro_economy",
                 embedding_model: Optional[str] = None):
        """初始化知识库检索器"""
        self.vector_db_path = Path(vector_db_path)
        self.json_dir_path = Path(json_dir_path)

        if embedding_model is None:
            import os
            embedding_model = os.getenv('EMBEDDING_MODEL_PATH', 'Qwen/Qwen3-Embedding-0.6B')

        self.embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
        self.vector_store = Chroma(
            collection_name="macro_economy_col",
            persist_directory=str(self.vector_db_path),
            embedding_function=self.embeddings
        )

        self._load_json_index()

    def _load_json_index(self):
        """加载JSON文件索引"""
        self.json_files = {}
        for json_file in self.json_dir_path.glob("*.json"):
            topic_num = int(json_file.name.split("_")[0])
            self.json_files[topic_num] = json_file

    def vector_search(self, query: str, k: int = 3) -> str:
        """向量检索"""
        results = self.vector_store.similarity_search(query, k=k)

        if not results:
            return "未找到相关知识"

        output = f"向量检索结果(共{len(results)}条):\n\n"
        for i, doc in enumerate(results, 1):
            output += f"[结果{i}]\n"
            output += f"内容: {doc.page_content[:200]}...\n"
            if doc.metadata:
                output += f"来源: 主题{doc.metadata.get('seq', 'N/A')} - {doc.metadata.get('topic', 'N/A')}\n"
            output += "\n"

        return output

    def get_topic_knowledge(self, topic_number: int) -> str:
        """按主题查询JSON知识"""
        if topic_number not in self.json_files:
            return f"错误: 主题{topic_number}不存在(有效范围1-17)"

        json_file = self.json_files[topic_number]
        with open(json_file, 'r', encoding='utf-8') as f:
            knowledge = json.load(f)

        output = f"主题{topic_number}: {knowledge.get('topic', 'N/A')}\n\n"

        if 'key_concepts' in knowledge:
            output += "关键概念:\n"
            for concept in knowledge['key_concepts'][:5]:
                name = concept.get('name', 'N/A')
                definition = concept.get('definition', 'N/A')
                output += f"  - {name}: {definition}\n"
            output += "\n"

        if 'indicators' in knowledge:
            output += "关键指标:\n"
            for indicator in knowledge['indicators'][:3]:
                name = indicator.get('name', 'N/A')
                interpretation = indicator.get('interpretation', 'N/A')
                calculation = indicator.get('calculation', 'N/A')
                output += f"  - {name}: {interpretation} | {calculation} \n"
            output += "\n"

        if 'summary' in knowledge:
            output += f"摘要:\n{knowledge['summary'][:300]}...\n"

        return output

print("[完成] 工具函数加载完成 (日志级别: WARNING)")

[完成] 工具函数加载完成 (日志级别: WARNING)


## Agent创建

**DeepAgents架构**：Main Agent + SubAgent

**SubAgent配置**：宏观经济分析专家，5个工具


In [ ]:
# 阶段2：宏观经济SubAgent实现（DeepAgent架构）

import os

# 加载环境变量
from dotenv import load_dotenv
load_dotenv('../config/.env')

from langchain_openai import ChatOpenAI
from deepagents import create_deep_agent

print("[初始化] 1/4 加载环境变量...", flush=True)

# 初始化DeepSeek模型
deepseek_model = ChatOpenAI(
    model="deepseek-chat",
    openai_api_key=os.getenv("DEEPSEEK_API_KEY"),
    openai_api_base="https://api.deepseek.com",
    streaming=True,
    temperature=0.7,
)

print("[初始化] 2/4 创建知识检索器...", flush=True)

# 初始化知识检索器
knowledge_retriever = KnowledgeRetriever()

print("[初始化] 3/4 包装知识检索工具...", flush=True)

# 包装知识检索方法为独立函数
def vector_search(query: str, k: int = 3) -> str:
    """向量检索知识库

    Args:
        query: 查询文本
        k: 返回结果数量(默认3)

    Returns:
        检索结果文本
    """
    print("[工具] 向量检索...", end="", flush=True)
    result = knowledge_retriever.vector_search(query, k)
    print(" 完成")
    return result

def get_topic_knowledge(topic_number: int) -> str:
    """按主题编号查询JSON知识

    Args:
        topic_number: 主题编号(1-17)

    Returns:
        主题知识内容
    """
    print(f"[工具] 查询主题{topic_number}...", end="", flush=True)
    result = knowledge_retriever.get_topic_knowledge(topic_number)
    print(" 完成")
    return result

print("[初始化] 4/4 创建DeepAgent+SubAgent...", flush=True)

# 定义宏观经济SubAgent配置（字典格式）
macroeconomic_subagent = {
    "name": "macroeconomic_subagent",
    "description": "负责宏观经济分析,包括GDP、CPI、PMI等指标的数据获取、分析和趋势判断。结合实时数据和理论知识提供专业分析。",
    "system_prompt": """你是宏观经济分析专家。

核心能力：数据获取、理论支撑、综合分析

输出要求：
- 数据准确，引用来源
- 分析专业，运用理论
- 逻辑清晰，结构完整
- 结论明确，便于理解""",
    "tools": [
        get_gdp_quarterly,
        get_cpi_monthly,
        get_pmi_manufacturing,
        vector_search,
        get_topic_knowledge,
    ],
}

# 创建Main Agent（使用create_deep_agent）
main_agent = create_deep_agent(
    model=deepseek_model,
    subagents=[macroeconomic_subagent],
)

print("[成功] DeepAgent+SubAgent架构已创建")
print(f"   类型: {type(main_agent).__name__}")
print(f"   模型: DeepSeek (deepseek-chat)")
print(f"   SubAgent: macroeconomic_subagent")
print(f"   工具数量: 5个")
print(f"     - AKShare工具: 3个 (GDP/CPI/PMI)")
print(f"     - 知识检索: 2个 (向量检索/JSON查询)")
print("\n[提示] Token级流式输出已启用，带详细进度追踪")

## 测试1：GDP分析

**测试目标**：验证数据获取+理论分析+综合判断能力

**进度追踪**：6个阶段（发送问题→Main Agent决策→SubAgent执行→返回结果→流式输出）


In [ ]:
# 测试1：GDP分析（DeepAgent详细进度追踪）
import time
from datetime import datetime
from langchain_core.messages import AIMessageChunk, ToolMessage

test_query_1 = "最近GDP增长率如何？有什么趋势特征？"

# 进度追踪
print("=" * 80)
print(f"[总进度] 测试1/3: GDP分析 (DeepAgent+SubAgent架构)")
print("=" * 80)
print(f"问题：{test_query_1}")
print("-" * 80)

start_time = time.time()
print(f"\n[{datetime.now().strftime('%H:%M:%S')}] 开始执行", flush=True)
print("[进度] 1/6 发送问题到Main Agent...", flush=True)

# 状态追踪
stage_times = {}
tool_call_count = 0
output_started = False
first_token_time = None
subagent_started = False

# 记录Main Agent决策时间
stage_times['main_agent_start'] = time.time()

# LangGraph的stream方法（字典输入 + stream_mode="messages"）
response_text = ""
main_agent_started = False

for message_chunk, metadata in main_agent.stream(
    {"messages": [{"role": "user", "content": test_query_1}]},
    stream_mode="messages"
):
    current_time = time.time()

    # 检测Main Agent开始决策（收到第一个消息）
    if not main_agent_started:
        print("[进度] 2/6 Main Agent决策中（分析问题，选择SubAgent）...", flush=True)
        main_agent_started = True

    # 检测工具调用（SubAgent执行）
    if isinstance(message_chunk, ToolMessage):
        if not subagent_started:
            elapsed = current_time - start_time
            print(f"\n[{datetime.now().strftime('%H:%M:%S')}] [进度] 3/6 Main Agent已决策，调用SubAgent (耗时{elapsed:.1f}s)", flush=True)
            print("[进度] 4/6 SubAgent执行中（调用工具+分析）...", flush=True)
            subagent_started = True
        tool_call_count += 1
        elapsed = current_time - start_time
        print(f"\n[{datetime.now().strftime('%H:%M:%S')}] 工具调用 {tool_call_count} 完成 (耗时{elapsed:.1f}s)", flush=True)

    # 检测AI消息（流式输出）
    elif isinstance(message_chunk, AIMessageChunk):
        # 检测首个token返回
        if message_chunk.content and not output_started:
            first_token_time = current_time - start_time
            print(f"\n[{datetime.now().strftime('%H:%M:%S')}] [进度] 5/6 SubAgent返回结果 (总延迟{first_token_time:.1f}s)", flush=True)
            print("[进度] 6/6 流式输出中...", flush=True)
            print(f"\n回答：", end="", flush=True)
            output_started = True

        # 输出流式内容
        if message_chunk.content:
            print(message_chunk.content, end="", flush=True)
            response_text += message_chunk.content

total_time = time.time() - start_time
print(f"\n\n[{datetime.now().strftime('%H:%M:%S')}] [完成]")
print("-" * 80)
print(f"[统计] 总耗时: {total_time:.2f}s | 工具调用: {tool_call_count}次 | 首token延迟: {first_token_time:.1f}s")
print("=" * 80)

## 测试2：通胀分析

**测试目标**：验证理论解读能力

**测试内容**：结合经济理论分析通胀水平


In [ ]:
# 测试2：通胀分析（带进度提示）
from langchain_core.messages import AIMessageChunk, ToolMessage

test_query_2 = "当前通胀水平怎么样？根据经济理论应该如何解读？"
print(f"问题：{test_query_2}", flush=True)
print("-" * 60, flush=True)
print("\n[Agent分析中...]", end="", flush=True)

tool_call_count = 0
output_started = False

for message_chunk, metadata in main_agent.stream(
    {"messages": [{"role": "user", "content": test_query_2}]},
    stream_mode="messages"
):
    if isinstance(message_chunk, ToolMessage):
        tool_call_count += 1
        print(f"\r[工具调用 {tool_call_count}/5]", end="", flush=True)

    elif isinstance(message_chunk, AIMessageChunk) and message_chunk.content:
        if not output_started:
            print("\r" + " " * 40, flush=True)
            print("\n回答：", end="", flush=True)
            output_started = True

        print(message_chunk.content, end="", flush=True)

print("\n")

## 测试3：PMI经济趋势分析

**测试目标**：验证周期理论应用能力

**测试内容**：结合经济周期理论分析PMI数据


In [ ]:
# 测试3：PMI经济趋势分析（带进度提示）
from langchain_core.messages import AIMessageChunk, ToolMessage

test_query_3 = "PMI数据显示经济趋势如何？结合经济周期理论分析"
print(f"问题：{test_query_3}", flush=True)
print("-" * 60, flush=True)
print("\n[Agent分析中...]", end="", flush=True)

tool_call_count = 0
output_started = False

for message_chunk, metadata in main_agent.stream(
    {"messages": [{"role": "user", "content": test_query_3}]},
    stream_mode="messages"
):
    if isinstance(message_chunk, ToolMessage):
        tool_call_count += 1
        print(f"\r[工具调用 {tool_call_count}/5]", end="", flush=True)

    elif isinstance(message_chunk, AIMessageChunk) and message_chunk.content:
        if not output_started:
            print("\r" + " " * 40, flush=True)
            print("\n回答：", end="", flush=True)
            output_started = True

        print(message_chunk.content, end="", flush=True)

print("\n")

## 批量测试准备

**测试内容**：按难度递增（基础查询→周期判断→投资策略→综合分析）


In [ ]:
# 10个测试场景（按难度递增）
test_questions = [
    # 基础数据查询（简单）
    "2024年GDP增长率是多少？",
    "当前的通胀水平如何？",
    "最新的PMI数据是多少？",

    # 周期判断（中等）
    "当前经济处于什么周期？",
    "经济周期转折的信号是什么？",
    "什么指标变化会预示周期转折？",

    # 投资策略（困难）
    "根据当前经济周期，应该配置什么资产？",
    "投资时钟当前处于哪个阶段？",

    # 综合分析（最难）
    "给出当前宏观经济的整体判断",
    "从宏观角度看，周期性行业投资机会如何？"
]

print(f"共{len(test_questions)}个测试问题，按难度递增")
for i, q in enumerate(test_questions, 1):
    print(f"{i}. {q}")

## 完整测试（10个场景）

**测试目标**：测试SubAgent的综合分析能力，评估输出质量

**测试方式**：取消注释运行批量测试脚本


In [ ]:
# 批量测试（注释状态，需要时取消下面的三引号运行）
"""
results = []

for i, question in enumerate(test_questions, 1):
    print(f"\n{'='*60}")
    print(f"测试 {i}/{len(test_questions)}: {question}")
    print('='*60)

    try:
        result = main_agent.invoke({"messages": [{"role": "user", "content": question}]})
        response_text = result["messages"][-1].content
        results.append({
            'question': question,
            'response': response_text,
            'status': 'success'
        })
        print(response_text)
    except Exception as e:
        results.append({
            'question': question,
            'error': str(e),
            'status': 'failed'
        })
        print(f"[错误] 错误: {e}")

# 保存测试结果
import json
with open('../data/outputs/stage2_test_results.json', 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print(f"\n[完成] 测试完成！成功: {sum(1 for r in results if r['status']=='success')}/{len(results)}")
"""

print("测试脚本已准备，取消三引号可运行")

## 输出质量评估标准

**数据准确性**（30分）：
- [必须] 引用正确来源（"根据AKShare最新数据...）
- [必须] 数据时间明确（"2024年11月..."）
- [必须] 数值准确无误

**分析专业性**（40分）：
- [必须] 运用理论框架（"根据经济周期理论..."）
- [必须] 分析逻辑清晰（数据→趋势→原因→影响）
- [必须] 结合知识库内容

**结论清晰度**（30分）：
- [必须] 给出明确判断（"当前处于XX周期"）
- [必须] 提出可行建议（"建议配置XX资产"）
- [必须] 易于理解（非专业人士能看懂）

**总分≥80分：优秀** | **60-79分：良好** | **<60分：需优化**
